In [7]:
import os
import cv2
import numpy as np
import dlib
from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb
from img_preprocessing import preprocess_images
from skimage.util import random_noise

%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
import cv2

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from eigenface import EigenFaces
from img_preprocessing import preprocess_images


import pickle

In [3]:
face_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)


predictor = dlib.shape_predictor("trials/shape_predictor_68_face_landmarks.dat")
fa = FaceAligner(predictor, desiredFaceWidth=250, desiredFaceHeight=250)


In [4]:
def align_images(path=None, output_path=None, resize=(250, 250)):        

    if path is None:
        path = os.path.join(os.getcwd(), 'images')
    
    if output_path is None:
        output_path = os.path.join(os.getcwd(), 'aligned_images')

        
    os.makedirs(output_path, exist_ok=True)

    
    images = []
    students = []

    for student in os.listdir(path):
        student_path = os.path.join(path, student)                
        
        if os.path.isdir(student_path):   
            os.makedirs(os.path.join(output_path, student), exist_ok=True)         
        
            for image_name in os.listdir(student_path):
                image_path = os.path.join(student_path, image_name)
        
                if os.path.isfile(image_path):
                    image = cv2.imread(image_path)
                    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                    
                    faces = face_classifier.detectMultiScale(
                        gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(40, 40)
                    )
                    
                    if len(faces) == 0:
                        print(image_path)
                        continue
                    
                    x, y, w, h = faces[0]

                    # face = cv2.resize(gray_image[y : y + h, x : x + w], (250, 250))

                    faceAligned = fa.align(image, gray_image, dlib.rectangle(x, y, x + h, y + w))

                    faceAligned = cv2.cvtColor(faceAligned, cv2.COLOR_BGR2GRAY)

                    img = preprocess_images([faceAligned])[0]               

                    images.append(img)
                    students.append(student)

                    output_image_path = os.path.join(output_path, student, image_name)
                    cv2.imwrite(output_image_path, img.reshape((250, 250)) * 255)

    return np.array(images), np.array(students)                                                             

In [5]:
images, students = align_images()


e:\4th-year\Semester-1\Mathematics-for-AI\Assignments\Project-FaceRecognition\images\Abraham Wendmeneh\8.jpg
e:\4th-year\Semester-1\Mathematics-for-AI\Assignments\Project-FaceRecognition\images\Biniyam Haile\3.jpg


In [8]:
x_train, x_test, y_train, y_test = train_test_split(images, students, test_size=0.2, random_state=42, stratify=students)

In [9]:
eigenface_model = EigenFaces(num_components=100).fit(x_train, y_train)

In [10]:
with open('models/eigenface_recognizer_aligned.pkl', 'wb') as f:
    pickle.dump(eigenface_model, f)

In [11]:
y_pred = eigenface_model.predict(x_test)

In [12]:
accuracy = accuracy_score(y_test, y_pred)

In [13]:
accuracy

0.9811320754716981